# Image Classification

In [ ]:
# importing libraries
import warnings
warnings.simplefilter('ignore')


import os
import cv2 
import math
import random
import shutil
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sb
import torch.nn as nn
import torch.nn.functional as F
from zipfile import ZipFile
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from keras.utils import to_categorical
from tqdm.notebook import tqdm 
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

## Data Preparation

In [ ]:
num_samples = 500
folder_path = r"animal classification"
data_path = os.path.join(folder_path, 'Data')
os.makedirs(data_path, exist_ok=True)

def unzip_data():
    for root, _, files in os.walk(folder_path):
        for file in files:
            path = os.path.join(root, file) 
            if path.endswith(".zip"):
                with ZipFile(path, 'r') as zf:
                    zf.extractall(path=data_path)
    print("Unzip Data Completed")

unzip_data()

Unzip Data Completed


In [ ]:
# building our dataset
def build_data():
    data = []
    for folder in os.listdir(data_path):
        if folder.startswith("train_"):
            target = folder.split("_")[-1]
            folder_path = os.path.join(data_path, folder)
            files_class = []
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpeg") and len(files_class) < num_samples:
                    file_path = os.path.join(folder_path, file_name)
                    files_class.append(file_path)
                    data.append({'image': file_name, 'filepath': file_path, 'target': target})
    return pd.DataFrame(data)


In [ ]:

data = build_data()
data = data.sample(frac=1).reset_index(drop=True)

data

,image,filepath,target
0,ASG0014ip9_2.jpeg,animal classification\Data\train_elephants\ASG...,elephants
1,ASG001e0in_2.jpeg,animal classification\Data\train_zebras\ASG001...,zebras
2,ASG001e0ep_0.jpeg,animal classification\Data\train_zebras\ASG001...,zebras
3,ASG0014jz5_1.jpeg,animal classification\Data\train_elephants\ASG...,elephants
4,ASG0014k2s_2.jpeg,animal classification\Data\train_elephants\ASG...,elephants
...,...,...,...
995,ASG001dwah_1.jpeg,animal classification\Data\train_zebras\ASG001...,zebras
996,ASG001e0mb_1.jpeg,animal classification\Data\train_zebras\ASG001...,zebras
997,ASG0014ipr_1.jpeg,animal classification\Data\train_elephants\ASG...,elephants
998,ASG001e0pp_2.jpeg,animal classification\Data\train_zebras\ASG001...,zebras


In [ ]:

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
data['class'] = label_encoder.fit_transform(data['target'])
data.head()

,image,filepath,target,class
0,ASG0014ip9_2.jpeg,animal classification\Data\train_elephants\ASG...,elephants,0
1,ASG001e0in_2.jpeg,animal classification\Data\train_zebras\ASG001...,zebras,1
2,ASG001e0ep_0.jpeg,animal classification\Data\train_zebras\ASG001...,zebras,1
3,ASG0014jz5_1.jpeg,animal classification\Data\train_elephants\ASG...,elephants,0
4,ASG0014k2s_2.jpeg,animal classification\Data\train_elephants\ASG...,elephants,0


In [ ]:
# splitting the data into train and validation set

class_samples = data.groupby('target').head(num_samples)
train_size = int(0.8 * len(class_samples))
train_dataset = class_samples[:train_size]
val_dataset = class_samples[train_size:]

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 800
Validation dataset size: 200


In [ ]:
# saving train and validation images into a folder

train_images_folder = os.path.join(data_path, 'train_images')
val_images_folder = os.path.join(data_path, 'val_images')
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)

# Save training  and validation images
def save_images(dataset, target_folder):
    for index, row in dataset.iterrows():
        image_path = row['filepath']
        target_class = row['target']
        class_folder = os.path.join(target_folder, target_class)
        os.makedirs(class_folder, exist_ok=True)
        shutil.copy(image_path, class_folder)


save_images(train_dataset, train_images_folder)
save_images(val_dataset, val_images_folder)

# Save files
train_csv_path=os.path.join(data_path,'train_dataset.csv')
val_csv_path=os.path.join(data_path,'val_dataset.csv')
train_dataset.to_csv(train_csv_path, index=False)
val_dataset.to_csv(val_csv_path, index=False)

print("Images and CSV files saved successfully.")


Images and CSV files saved successfully.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_frame.iloc[idx, 1])
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, 3]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:
# Define transformations and dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomDataset(csv_file=train_csv_path, root_dir=train_images_folder, transform=transform)
val_dataset = CustomDataset(csv_file=val_csv_path, root_dir=val_images_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



## Model Building

In [ ]:
num_classes=len(np.unique(data['target']))
num_classes

2

In [ ]:
# model architecture
class SimpleModel(nn.Module):
    def __init__(self, num_classes, last_units=64, conv_kernel_size=5,dropout=0.3):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=conv_kernel_size, stride=1, padding=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=conv_kernel_size, stride=1, padding=2)
        self.fc1 = nn.Linear(16 * 56 * 56, last_units)  
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(last_units, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x)) 
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16 * 56 * 56)
        x = self.dropout(x)
        x = F.sigmoid(self.fc1(x)) 
        x = self.fc2(x)
        return x

## Training model without hyperparameter tuning

In [ ]:
# Initialize model

model = SimpleModel(num_classes=2, last_units=64,conv_kernel_size=5,dropout=0.1)  

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:


# Train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)  
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.8612
Epoch [2/10], Loss: 0.7145
Epoch [3/10], Loss: 0.6984
Epoch [4/10], Loss: 0.6932
Epoch [5/10], Loss: 0.7011
Epoch [6/10], Loss: 0.6936
Epoch [7/10], Loss: 0.6960
Epoch [8/10], Loss: 0.7155
Epoch [9/10], Loss: 0.7176
Epoch [10/10], Loss: 0.7065


In [ ]:
# Model evaluation
def evaluate_model(model, val_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    print(f"Validation Accuracy: {val_accuracy:.4f}")


evaluate_model(model, val_loader)

Validation Accuracy: 0.5050


## Model training and evaluation with hyperparameter tuning

In [ ]:


# Define the Skorch classifier with the given PyTorch model
net = NeuralNetClassifier(
    module=SimpleModel,
    module__num_classes=num_classes,  
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    optimizer__lr=0.001,
    batch_size=32,
    max_epochs=10
)

# Extract data and labels  from the train loader
X_train = []
y_train = []
for images, labels in train_loader:
    X_train.append(images.numpy())  
    y_train.append(labels.numpy())  
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Defining the hyperparameters 
params = {
    'module__dropout': [0.1, 0.4, 0.6],
    'module__conv_kernel_size': [2,3,4],
    'module__last_units': [64, 128]

}

# Perform grid search with cross-validation
gs = GridSearchCV(net, params, cv=3, scoring='accuracy', verbose=1)

# Train the model
gs.fit(X_train, y_train)


print("Best parameters found: ", gs.best_params_)
print("Best accuracy found: ", gs.best_score_)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.4229       0.9252        0.2247  4.8486
      2        0.1927       0.9439        0.1974  5.3318
      3        0.1563       0.9346        0.1804  5.4840
      4        0.1536       0.9533        0.1639  4.8717
      5        0.1343       0.9626        0.1469  4.6400
      6        0.1271       0.9626        0.1399  4.5206
      7        0.1222       0.9626        0.1367  4.6700
      8        0.1193       0.9626        0.1374  4.5189
      9        0.1177       0.9626        0.1358  4.7602
     10        0.1165       0.9626        0.1360  4.8338
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.0847       0.4953        0.9508  4.6394
      2        0.8671       0.4953        0.7909  4.6250
      3        0.7479      

      4        0.1320       0.9533        0.1658  4.3898
      5        0.1250       0.9533        0.1599  4.5248
      6        0.1194       0.9533        0.1476  4.6308
      7        0.1099       0.9626        0.1377  4.9268
      8        0.1047       0.9626        0.1230  4.5814
      9        0.0950       0.9626        0.1146  4.6105
     10        0.0859       0.9626        0.1065  4.8628
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.5757       0.8785        0.2803  4.7998
      2        0.1852       0.9533        0.1604  4.8336
      3        0.1291       0.9533        0.1122  4.3989
      4        0.0801       0.9720        0.0783  4.8420
      5        0.0503       0.9813        0.0733  4.6510
      6        0.0309       0.9813        0.0630  4.5635
      7        0.0173       0.9907        0.0356  4.7827
      8        0.0143       0.9907        0.0558  4.7543
      9        0.0106       0.9

     10        0.0017       0.9938        0.0243  6.6642
Best parameters found:  {'module__conv1_kernel_size': 4, 'module__dropout': 0.4, 'module__last_units': 128}
Best accuracy found:  0.9699858259881915


In [ ]:
best_net = gs.best_estimator_

scores=[]
with torch.no_grad():

    for images, labels in val_loader:
        score= best_net.score(images,labels)
        scores.append(score)
print(f"Validation Accuracy after Hyper parameter tuning: {np.mean(scores)}")

Validation Accuracy after Hyper parameter tuning: 0.9598214285714286
